In [ ]:
%pip install qiskit torch numpy pandas

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Function
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from sklearn.decomposition import PCA

#Prep Training Data (Breast Cancer Wisconsin)
def extract_expval(pub_result):
    """Return a plain Python float for the first observable/parameter set,
       whatever Qiskit version is installed."""
    if hasattr(pub_result, "values"):           # 0.46-style Estimator V1
        return float(pub_result.values[0])
    elif hasattr(pub_result, "data"):           # 0.47+ Estimator V2
        return float(pub_result.data.evs.item())     # .item() -> scalar
    else:
        raise TypeError("Unrecognised estimator result structure.")

#Load the data file
df = pd.read_csv("data\\wdbc.data", header=None)

#Assign column names
columns = ['id', 'diagnosis'] + [f'feature_{i}' for i in range(1, 31)]
df.columns = columns

#Drop the ID column
df = df.drop(columns=['id'])

#Encode diagnosis: M = 1, B = 0
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

#Convert to numpy arrays
X = df.drop(columns=['diagnosis']).values.astype(np.float32)
Y = df['diagnosis'].values.astype(np.float32).reshape(-1, 1)

#Normalize features manually (z-score)
mean = X.mean(axis=0)
std = X.std(axis=0)
X = (X - mean) / std
pca = PCA(n_components=2, whiten=True, random_state=1)
X = pca.fit_transform(X).astype(np.float32)

#Convert to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)

#Manual train/test split (80/20)
num_samples = X_tensor.shape[0]
indices = torch.randperm(num_samples)

split_idx = int(num_samples * 0.8)
train_indices = indices[:split_idx]
test_indices = indices[split_idx:]

X_train = X_tensor[train_indices]
Y_train = Y_tensor[train_indices]
X_test = X_tensor[test_indices]
Y_test = Y_tensor[test_indices]

#VQA Circuit
n_qubits = 2
params = ParameterVector('theta', length=4)


def create_vqa_circuit(inputs, weights):
    """
    inputs  : 1-D numpy array of length 2  (classical features)
    weights : 1-D numpy array of length 4  (trainable parameters)
    """
    qc = QuantumCircuit(n_qubits)

    # --- feature encoding (one RY per qubit) -------------
    for q, x in enumerate(inputs):
        qc.ry(float(x), q)

    # --- simple entanglement -----------------------------
    qc.cx(0, 1)

    # --- variational layer -------------------------------
    qc.ry(float(weights[0]), 0)
    qc.rz(float(weights[1]), 0)
    qc.ry(float(weights[2]), 1)
    qc.rz(float(weights[3]), 1)

    return qc


#Qiskit StatevectorEstimator primitive
estimator = StatevectorEstimator()
observables = [SparsePauliOp("ZI")]

#PyTorch Custom Autograd Function For VQA Layer
# --- 2. Autograd function with parameter-shift -----------

class VQALayerFunction(Function):
    @staticmethod
    def forward(ctx, input_tensor, weights):
        ctx.save_for_backward(input_tensor, weights)

        pub = (create_vqa_circuit(input_tensor.numpy(),
                                  weights.detach().numpy()),
               SparsePauliOp("ZI"))
        job     = estimator.run([pub])          # ← reuse the global estimator
        expval  = extract_expval(job.result()[0])

        return torch.tensor([expval],
                            dtype=torch.float32,
                            device=input_tensor.device)

    @staticmethod
    def backward(ctx, grad_output):
        input_tensor, weights = ctx.saved_tensors
        x   = input_tensor.detach().numpy()
        th0 = weights.detach().numpy()
        sh  = np.pi / 2

        grads = np.zeros_like(th0, dtype=np.float32)

        for i in range(len(th0)):
            th_plus, th_minus = th0.copy(), th0.copy()
            th_plus[i]  += sh
            th_minus[i] -= sh

            pub_plus  = (create_vqa_circuit(x, th_plus),  SparsePauliOp("ZI"))
            pub_minus = (create_vqa_circuit(x, th_minus), SparsePauliOp("ZI"))

            exp_plus  = extract_expval(estimator.run([pub_plus ]).result()[0])
            exp_minus = extract_expval(estimator.run([pub_minus]).result()[0])

            grads[i] = 0.5 * (exp_plus - exp_minus)

        grad_weights = grad_output.view(-1)[0] * torch.tensor(
            grads, dtype=torch.float32, device=weights.device
        )

        return None, grad_weights



#Quantum Layer as PyTorch Module
class VQALayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(4)) #4 trainable parameters for the circuit

    def forward(self, x):
        return torch.stack([VQALayerFunction.apply(x[i], self.weights) for i in range(x.size(0))]).view(-1, 1)

#Full Hybrid Model
class HybridModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.classical = nn.Linear(X_tensor.shape[1], 2) #Classic preprocessing layer
        self.quantum = VQALayer() #VQA layer
        self.output = nn.Linear(1, 1) #Final classical layer

    def forward(self, x):
        x = self.classical(x)
        x = torch.tanh(x)  #Activation before quantum layer
        x = self.quantum(x)
        x = self.output(x)
        return x

model = HybridModel()
pos_weight = torch.tensor([Y_train.mean().reciprocal()]) 
loss_fn    = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
sigmoid = torch.nn.Sigmoid()
# loss_fn = nn.BCELoss()
#Training Loop
# for epoch in range(50):
#     optimizer.zero_grad()
#     preds = model(X_train)
#     loss = loss_fn(preds, Y_train)
#     loss.backward()
#     optimizer.step()
#     with torch.no_grad():
#         acc = ((preds > 0.5).float() == Y_train).float().mean()
#         print(f"Epoch {epoch+1} | Loss: {loss.item():.4f} | Accuracy: {acc.item()*100:.2f}%")

for epoch in range(70):
    # ----------- train on the whole set (no mini-batch) ------------------
    model.train()
    optimizer.zero_grad()
    logits = model(X_train)
    loss   = loss_fn(logits, Y_train)
    loss.backward()
    optimizer.step()

    # training accuracy
    with torch.no_grad():
        train_pred = sigmoid(logits) > 0.5
        train_acc  = (train_pred == Y_train).float().mean().item() * 100

    # ------------------- evaluate on the test set ------------------------
    model.eval()
    with torch.no_grad():
        test_logits = model(X_test)
        test_pred   = sigmoid(test_logits) > 0.5
        test_acc    = (test_pred == Y_test).float().mean().item() * 100
        test_loss   = loss_fn(test_logits, Y_test).item()

    print(f"Epoch {epoch+1:02d}  "
          f"Train-loss {loss.item():.4f}  Train-acc {train_acc:5.1f}%   "
          f"Test-loss {test_loss:.4f}  Test-acc {test_acc:5.1f}%")



KeyboardInterrupt: 